In [ ]:
!pip install --upgrade langchain openai
!pip install tiktoken
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement llamba-index (from versions: none)
ERROR: No matching distribution found for llamba-index
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 54.6 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os


In [ ]:

# args
os.environ["OPENAI_API_KEY"] = "API_KEY\nlocal_repo_path = '/content/dhSegment'
readthedocs_question = ['url array']
wand_readthedocs = ['url array']


In [ ]:
!git clone https://github.com/dhlab-epfl/dhSegment.git

Cloning into 'dhSegment'...
remote: Enumerating objects: 2536, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2536 (delta 0), reused 1 (delta 0), pack-reused 2530
Receiving objects: 100% (2536/2536), 6.04 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (1637/1637), done.


In [ ]:
# add repo_in_question

repo_path = local_repo_path

docs = []
for dirpath, dirnames, filenames in os.walk(repo_path):
    for file in filenames:
        if file.endswith('.py') and '/.venv/' not in dirpath:
            try: 
                loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
                docs.extend(loader.load_and_split())
            except Exception as e: 
                pass
print(f'{len(docs)}')

94


In [ ]:
# add readthedocs_in_question



In [ ]:
# add wandb_docs_helper


In [ ]:
# combine repo_in_question, readthedocs_in_question, wandb_docs_helper


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [ ]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)


In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 20
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 20

In [ ]:
model = ChatOpenAI(model_name='gpt-3.5-turbo')
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [ ]:
questions = [
    "how do i extract text lines with dhsegment",
    "give me an example of how to do this"
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: how do i extract text lines with dhsegment 

**Answer**: The dhsegment package provides a framework for historical document processing. To extract text lines with dhsegment, you can use the `page_extraction` function provided in the code you have. This function takes an input image and outputs the text lines found in the image in a PAGEXML file format.

Here are the steps you can follow to use `page_extraction`:

1. Install dhsegment by running `pip install dhsegment` in your terminal.
2. Import the `page_extraction` function by adding `from dh_segment.post_processing import page_extraction` at the top of your Python file.
3. Call `page_extraction` with the following arguments:
   - `model_dir`: The directory of the model to use for text line extraction.
   - `filenames_to_process`: A list of image file paths to extract text lines from.
   - `output_dir`: The directory to output the PAGEXML files.
   - `draw_extractions`: If true, the extracted lines will be drawn and 

In [ ]:
"""
bib:

langchain_docs:
  git_loader: https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/git.html
  readthedocs_loader: https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/readthedocs_documentation.html

  
"""